In [39]:
import subprocess
import time
import datetime
import collections
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

In [3]:
proc = subprocess.Popen(['echo', 'Hello world from child.'],stdout=subprocess.PIPE)

In [4]:
out, err = proc.communicate()

In [5]:
out

b'Hello world from child.\n'

In [14]:
proc = subprocess.Popen(['sleep', '0.01'])
while proc.poll() is None:
    print('working...')
    for i in range(10):
        print(i)

working...
0
1
2
3
4
5
6
7
8
9
working...
0
1
2
3
4
5
6
7
8
9


In [15]:
def run_sleep(period):
    proc = subprocess.Popen(['sleep', str(period)])
    return proc

In [46]:
start = time.time()
procs = []
for _ in range(10):
    proc = run_sleep(0.1)
    procs.append(proc)

for proc in procs:
    ## 子プロセスが終了するまで待つ
    proc.communicate()
end = time.time()
print('Finished in %.3f seconds' % (end - start))

Finished in 0.220 seconds


In [47]:
proc = subprocess.Popen(['pwd'],stdout=subprocess.PIPE)

out = proc.communicate()

print(out)

(b'/home/jovyan/work\n', None)


In [5]:
def sample_func(process_index):
    start = time.time()
    print('process index: %s started.' % process_index)
    num = 0

    for i in range(10000000):
        num += 1
    end = time.time()
    print('process index: %s ended.' % process_index)
    print('%s' % (end - start))

In [6]:
# %%timeit
# for i in range(10):
#     sample_func(process_index=0)

In [7]:
sample_func(0)

process index: 0 started.
process index: 0 ended.
1.0255699157714844


In [18]:
from multiprocessing import Manager, Process

def sample_func(initial_num, returned_dict, process_index):
    for i in range(100):
        initial_num += 1
    returned_dict[process_index] = initial_num


manager = Manager()
returned_dict = manager.dict()
process_list = []

for i in range(100):
    process = Process(
        target=sample_func,
        kwargs={
            'initial_num': 50,
            'returned_dict': returned_dict,
            'process_index': i,
        })

    process.start()
process.join()

In [30]:
# ライフゲーム
Query = collections.namedtuple('Query', ('y', 'x'))

In [98]:
def gcd(pair):
    a, b = pair
    if a < b:
        low = a
    else:
        low = b
    print("a: %f b: %f" %(a, b))
    for i in range(int(low), 0, -1):
        if a % i == 0 and b % i ==0:
            return i

In [125]:
start = time.time()
results = list(map(gcd, arr_list))
end = time.time()
print('Took %.3f seconds' %( end - start))

a: 865501.152472 b: 475718.310627
a: 683905.888852 b: 267933.901488
a: 646655.971265 b: 306217.037084
a: 841853.247549 b: 270599.702898
a: 403038.653798 b: 188550.641235
a: 748448.801167 b: 871312.356173
a: 197063.590462 b: 317559.867855
a: 603294.972299 b: 187911.455068
a: 215916.724752 b: 611757.523504
a: 906412.055810 b: 760885.341313
Took 4.246 seconds


In [126]:
start = time.time()
pool = ThreadPoolExecutor(max_workers=2)
results = list(pool.map(gcd, arr_list))
end = time.time()
print('Took %.3f seconds' %( end - start))

a: 865501.152472 b: 475718.310627
a: 683905.888852 b: 267933.901488
a: 646655.971265 b: 306217.037084
a: 841853.247549 b: 270599.702898
a: 403038.653798 b: 188550.641235
a: 748448.801167 b: 871312.356173
a: 197063.590462 b: 317559.867855
a: 603294.972299 b: 187911.455068
a: 215916.724752 b: 611757.523504
a: 906412.055810 b: 760885.341313
Took 4.641 seconds


In [124]:
start = time.time()
pool = ProcessPoolExecutor(max_workers=10)
results = list(pool.map(gcd, arr_list))
end = time.time()
print('Took %.3f seconds' %( end - start))

a: 748448.801167 b: 871312.356173
a: 197063.590462 b: 317559.867855
a: 603294.972299 b: 187911.455068
a: 403038.653798 b: 188550.641235
a: 841853.247549 b: 270599.702898
a: 906412.055810 b: 760885.341313
a: 865501.152472 b: 475718.310627
a: 215916.724752 b: 611757.523504
a: 646655.971265 b: 306217.037084
a: 683905.888852 b: 267933.901488
Took 2.816 seconds


In [59]:
results

[2, 1, 1]

In [87]:
import numpy as np

rand = np.random.rand()  # 一つの乱数
print(rand)
# 0.5399711080561779

arr = np.random.rand(2)  # サイズが3の配列の乱数
print(arr)
# [ 0.22393312  0.56723404  0.41167989]

0.20427879740011823
[0.67433001 3.57242506]


In [82]:
a = tuple([np.round(arr * 100000), arr])

In [123]:
arr_list = []
for value in range(10):
    arr_list.append(np.random.rand(2) * 100000)

In [105]:
int(arr_list[0][0])

3000